In [3]:
from dotenv import load_dotenv
import os
from langchain_core.prompts import PromptTemplate 
from langchain_core.language_models.llms import LLM
from langchain_groq.chat_models import ChatGroq
import json
from langgraph.graph import StateGraph, START, END
from typing_extensions import TypedDict

In [5]:
load_dotenv()

GROQ_API_KEY = os.getenv("GROQ_API_KEY")

In [6]:
# Prompt template for extracting topic, tone, and word count
extract_prompt = """
You are an assistant that extracts information from user input. Your task is to identify:
1. The main topic.
2. The tone of the request (e.g., formal, casual, persuasive, etc.).
3. The desired word count (if explicitly mentioned, otherwise provide a default value of 800).

User input: "{user_input}"

Provide your response in JSON format:
{
    "topic": "<main topic>",
    "tone": "<identified tone>",
    "word_count": <word count>
}
"""

input_prompt = PromptTemplate(
    input_variables = ['user_input'],
    template = extract_prompt,
)



#   Prompt Template
blog_template = """
You are a professional blog writer. Write a comprehensive blog on the topic:
"{topic}"

Make sure the blog includes:
1. A catchy introduction.
2. Informative body paragraphs with subheadings.
3. A conclusion with a call to action.

The tone should be {tone}.
Word count: {word_count} words.
"""

generate_prompt = PromptTemplate(
    input_variables = ['topic', 'tone', 'word_count'],
    template = blog_template
)

#   Set up LLM
llm = ChatGroq(model = "mixtral-8x7b-32768", temperature = 0.7,api_key = GROQ_API_KEY)


#   Creating a chain combining LLM and prompt
extraction_chain = input_prompt | llm
generation_chain = generate_prompt | llm

In [ ]:
class AgentState(TypedDict):
    """

    Attributes:
        question: question
        generation: LLM generated blog
    """

    question: str
    response : str

In [59]:
def extract_information(state:AgentState):
    """
    Extracts the topic, tone and wordcount from the user input

    Args:
        state (messages): The current state

    Returns:
        dict: The updated state with the agent response appended to messages
    """

    
    question = state["question"]
    print(question)
    response = extraction_chain.invoke({"user_input": question})
    
    try:
        # Parse the JSON string to a dictionary
        extracted_info = json.loads(response)
        # Extract variables
        topic = extracted_info.get("topic", "Unknown")
        tone = extracted_info.get("tone", "Neutral")
        word_count = extracted_info.get("word_count", 800)  # Default to 800 if missing
        return topic, tone, word_count
    except json.JSONDecodeError:
        print("Failed to parse response as JSON.")
        return None, None, None





In [60]:
def generate_blog(state:AgentState):

    """
    Writes up a blog as instructed by the user
    
    """
    return generation_chain.invoke({
        "topic":state['topic'],
        "tone":state['tone'],
        "word_count":state['word_count']
    })


In [61]:
graph_builder = StateGraph(AgentState)

In [62]:
graph_builder.add_node("Extractor", extract_information)
graph_builder.add_node("Generator", generate_blog)

graph_builder.add_edge(START,"extract_information")
graph_builder.add_edge("extract_information","generate_blog")
graph_builder.add_edge("generate_blog",END)
graph = graph_builder.compile


In [63]:
from IPython.display import Image, display

try:
     display(Image(graph_builder.get_graph().draw_mermaid_png()))
except Exception:
     # This requires some extra dependencies and is optional
     print("No graph")
     

No graph


In [64]:
def stream_graph_updates(user_input: str):
    for event in graph.stream({"question": user_input}):
        for value in event.values():
          if "response" in value:
            # print(type(value['response']))
            print("Assistant:", value["response"].content)

while True:
        user_input = input("User :")
        if user_input in ['q','quit','exit']:
            print("GoodBye")
            break
        topic,tone,word_count = extract_information(user_input)

        blog_content = generate_blog(
            topic = topic,
            tone = tone,
            word_count = word_count
        )

        print(blog_content.content)

TypeError: string indices must be integers, not 'str'

In [65]:
extract_template = """
You are an assistant that extracts information from user input. Your task is to identify:
1. The main topic.
2. The tone of the request (e.g., formal, casual, persuasive, etc.).
3. The desired word count (if explicitly mentioned, otherwise provide a default value of 800).

User input: "{user_input}"

Provide your response in JSON format:
{
    "topic": "<main topic>",
    "tone": "<identified tone>",
    "word_count": <word count>
}
"""

In [1]:
extract_prompt = PromptTemplate(
    input_variables= ['user_input'],
    template=extract_template
)

NameError: name 'PromptTemplate' is not defined

In [ ]:
extraction_chain = extract_prompt | llm

In [69]:
response = extraction_chain.invoke({"user_input": "Write a fun blog on dogs in 100 words"})

KeyError: 'Input to PromptTemplate is missing variables {\'\\n    "topic"\'}.  Expected: [\'\\n    "topic"\', \'user_input\'] Received: [\'user_input\']\nNote: if you intended {\n    "topic"} to be part of the string and not a variable, please escape it with double curly braces like: \'{{\n    "topic"}}\'.\nFor troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/INVALID_PROMPT_INPUT '

In [ ]:
blog_template = """
You are a professional blog writer. Write a comprehensive blog on the topic:
"{topic}"

Make sure the blog includes:
1. A catchy introduction.
2. Informative body paragraphs with subheadings.
3. A conclusion with a call to action.

The tone should be {tone}.
Word count: {word_count} words.
"""

generate_prompt = PromptTemplate(
    input_variables = ['topic', 'tone', 'word_count'],
    template = blog_template
)


In [ ]:
generation_chain = generate_prompt | llm

In [ ]:
result =  generation_chain.invoke({
        "topic":state['topic'],
        "tone":state['tone'],
        "word_count":state['word_count']
})